In [7]:
from utils import read_images, return_stacks_per_position_id, make_isotropic, extract_nuclei_labels, measure_intensity
import pandas as pd

In [8]:
# Define the path containing your stacks, the position id you want to explore and the scaling in x, y and z.
data_path = "./raw_data/ANP32A_N4/Crop"
position_id = 1
scaling_x_um = 0.342
scaling_y_um = 0.342
scaling_z_um = 0.663

# Define the analysis parameters
top_hat_radius = 5
gaussian_sigma = 2
voronoi_otsu_spot_sigma = 10
voronoi_otsu_outline_sigma = 1
closing_labels_radius = 5
erosion_labels_radius = 3

In [9]:
# Retrieve the paths of all the images within data_path (ignoring ch02, brightfield channel)
images_per_position = read_images(data_path)

# Initialize an empty list to store per position_id dataframes
dataframes = []

for position_id, files in images_per_position.items():
    
    # Retrieve the paths of all the images within data_path (ignoring ch02, brightfield channel)
    images_per_position = read_images(data_path)

    # Generate the ch00_stack (nuclei) and ch01_stack (protein of interest) from the individual slices 
    ch00_stack, ch01_stack = return_stacks_per_position_id(images_per_position, position_id)

    # Extract the nuclei labels from the ch00_stack
    nuclei_labels = extract_nuclei_labels(ch00_stack, 
                            scaling_x_um, 
                            scaling_y_um, 
                            scaling_z_um, 
                            top_hat_radius, 
                            gaussian_sigma, 
                            voronoi_otsu_spot_sigma,
                            voronoi_otsu_outline_sigma,
                            closing_labels_radius,
                            erosion_labels_radius)

    # Rescale the ch01_stack (protein of interest) to make data isotropic in order to include it as intensity_image 
    marker_resampled = make_isotropic(ch01_stack, scaling_x_um, scaling_y_um, scaling_z_um)

    # Extract intensity stats on a per nucleus basis
    df = measure_intensity(nuclei_labels, marker_resampled)

    # Insert position_id into the dataframe
    df.insert(0, 'position_id', position_id)

    # Append the per_position df to the dataframes list
    dataframes.append(df)

# Concatenate all DataFrames in the list
concatenated_df = pd.concat(dataframes, ignore_index=True)

concatenated_df

,position_id,label,intensity_mean,intensity_max,centroid-0,centroid-1,centroid-2,area_filled
0,1,1,12.378336,255.0,20.571527,374.311348,25.074347,14540.0
1,1,2,4.317102,65.0,18.783799,88.222357,34.940317,9098.0
2,1,3,9.201468,255.0,21.138459,542.593574,49.092813,11173.0
3,1,4,19.957376,255.0,12.472107,646.237623,122.217171,9877.0
4,1,5,8.319921,255.0,15.055299,49.048593,142.216689,9096.0
...,...,...,...,...,...,...,...,...
309,8,32,11.219877,255.0,19.955287,332.652311,566.305564,38244.0
310,8,33,4.521918,94.0,21.946929,678.544615,573.884068,8988.0
311,8,34,4.379683,188.0,14.564179,636.799521,582.992308,10011.0
312,8,35,5.614121,255.0,20.242161,606.220190,691.421662,18752.0
